In [1]:
import itertools
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as D
from torch.utils.data import DataLoader, TensorDataset
global  samplesJoint,tp_dataloader,tp_dataloader_testing,dp,decision,dp_H,decision_H
n = 5
epochs = 2
supervisionEpochs = 2
lr = 0.001
log_interval = 20
trainSize = 10000
penaltyLambda = 10
doublePeakHighMean = 0.9
doublePeakLowMean = 0.1
doublePeakStd = 0.1
uniformlow=0
uniformhigh=1.0
normalloc = 0.2
normalscale = 0.1

#stage=["twopeak","normal","uniform"]
stage=["independent"]
#order1name=["costsharing","dp","heuristic","random initializing"]
order1name=["costsharing"]

In [2]:
d1 = D.normal.Normal(doublePeakLowMean, doublePeakStd)
d2 = D.normal.Normal(doublePeakHighMean, doublePeakStd)
distributionRatio = (d1.cdf(1) + d2.cdf(1) - d1.cdf(0) - d2.cdf(0)) / 2
distributionBase = d1.cdf(0) + d2.cdf(0)

d3 = D.normal.Normal(normalloc, normalscale)
d4 = D.uniform.Uniform(uniformlow,uniformhigh)


def cdf(x,y, i=None):
    if(y=="twopeak"):
        return (d1.cdf(x) + d2.cdf(x) - distributionBase) / 2 / distributionRatio
    elif(y=="normal"):
        return d3.cdf(x);
    elif(y=="uniform"):
        return d4.cdf(x);

In [3]:
def bitsToPayments(bits):
    if sum(bits).item() == 0:
        return torch.ones(n)
    bits = bits.type(torch.float32)
    negBits = torch.ones(n) - bits
    payments = model(bits)
    payments = payments - 1000 * negBits
    payments = torch.softmax(payments, 0)
    payments = payments + negBits
    return payments

def tpToBits(tp, bits=torch.ones(n).type(torch.uint8)):
    payments = bitsToPayments(bits)
    newBits = (tp >= payments).type(torch.uint8)
    if torch.equal(newBits, bits):
        return bits
    else:
        return tpToBits(tp, newBits)#bits-bits#tpToBits(tp, newBits)

def tpToPayments(tp):
    return bitsToPayments(tpToBits(tp))


def tpToTotalDelay(tp):
    return n - sum(tpToBits(tp).type(torch.float32))


In [4]:
def dpSupervisionRule(tp):
    global  dp,decision
    tp = list(tp.numpy())
    bits = [1 for ii in range(n)]
    payments = [0.0 for ii in range(n)]
    money = dpPrecision
    yes = 0
    for i in range(n):
        offerIndex = decision[n - i, money, yes]
        offer = float(offerIndex) / dpPrecision
        if tp[i] >= offer:
            money -= offerIndex
            yes += 1
            bits[i] = 1
            payments[i] = offer
        else:
            bits[i] = 0
            payments[i] = 1
    if money > 0:
        bits = [0 for ii in range(n)]
        payments = [1.0 for ii in range(n)]
    bits = torch.tensor(bits, dtype=torch.uint8)
    payments = torch.tensor(payments, dtype=torch.float32)
#     print()
#     print(tp)
#     print(bits)
#     print(payments)
#     print()
    return (bits, payments)


def heuristicSupervisionRule(tp):
    global  dp_H,decision_H
    tp = list(tp.numpy())
    bits = [1 for ii in range(n)]
    payments = [0.0 for ii in range(n)]
    tempold=-1;
    for turns in range(n,0,-1):
        money = dpPrecision
        j=0
        tempo=sum(bits)
        #print("bits",tempo)
        for i in range(n):
            if(j>=n):
                break;
            
            offerIndex = decision_H[tempo ,tempo - i, money]
            offer = float(offerIndex) / dpPrecision
            while(j<n):
                if(bits[j]!=0):
                    break;
                j+=1;
            if(j>=n):
                break;
            if tp[j] >= offer:
                #print(money,j,tp[j],offer)
                money -= offerIndex
                bits[j] = 1
                payments[j] = offer
            else:
                bits[j] = 0;
                payments[j] = 1.0;
            j+=1;
        #print("money",money)
        if(money==0 and tempold==tempo):
            break;
        tempold=tempo;
    if money > 0:
        bits = [0 for ii in range(n)]
        payments = [1.0 for ii in range(n)]

    bits = torch.tensor(bits, dtype=torch.float32)
    payments = torch.tensor(payments, dtype=torch.float32)
    # print()
    # print(tp)
    # print(bits)
    # print(payments)
    # print()
    return (bits, payments)



def costSharingSupervisionRule(tp):
    tp = list(tp.numpy())
    for k in range(n, -1, -1):
        if k == 0:
            break
        bits = [1 if tp[ii] >= 1.0 / k else 0 for ii in range(n)]
        if sum(bits) == k:
            break
    if k == 0:
        payments = [1 for ii in range(n)]
    else:
        payments = [1.0 / k if bits[ii] == 1 else 1 for ii in range(n)]
    bits = torch.tensor(bits, dtype=torch.uint8)
    payments = torch.tensor(payments, dtype=torch.float32)
    return (bits, payments)


def costSharingDelay(tp):
    return float(n - sum(costSharingSupervisionRule(tp)[0]).item())

def dpDelay(tp):
    return float(n - sum(dpSupervisionRule(tp)[0]).item())

def heuristicDelay(tp):
    return float(n - sum(heuristicSupervisionRule(tp)[0]).item())


# templire=0;
# num=1
# for i in range(num):
#     temp=torch.tensor(samplesJoint[i])
#     #print(temp)
#     #print(temp);
#     #print(heuristicSupervisionRule(temp))
#     #print(dpSupervisionRule(temp))
#     #print(dpDelay(temp))
#     res=dpDelay(temp)
#     templire+=res
#     #print("delay",res)
#     #print()
# print(templire/num)

In [5]:
def recordAndReport(name, source, loss, n=100):
    source.append(loss)
    realLength = min(n, len(source))
    avgLoss = sum(source[-n:]) / realLength
    #print(f"{name} ({realLength}): {avgLoss}")
    print(name,realLength,":" ,avgLoss)


def supervisionTrain(epoch, supervisionRule):
    model.train()
    for batch_idx, (tp_batch,) in enumerate(tp_dataloader):
        optimizer.zero_grad()
        loss = 0
        for tp in tp_batch:
            bits, payments = supervisionRule(tp)
            loss = loss + F.mse_loss(bitsToPayments(bits), payments)

        loss = loss / len(tp_batch)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(tp_batch),
                    len(tp_dataloader.dataset),
                    100.0 * batch_idx / len(tp_dataloader),
                    loss.item(),
                )
            )


def train(epoch):
    model.train()
    for batch_idx, (tp_batch,) in enumerate(tp_dataloader):
        optimizer.zero_grad()
        penalty = 0
        for bitsMoreOnes in allBits:
            for i in range(n):
                if bitsMoreOnes[i] == 1:
                    bitsLessOnes = bitsMoreOnes.clone()
                    bitsLessOnes[i] = 0
                    penalty = penalty + sum(
                        torch.relu(
                            bitsToPayments(bitsMoreOnes) - bitsToPayments(bitsLessOnes)
                        )
                    )
        loss = penalty * penaltyLambda
        for tp in tp_batch:
            for i in range(n):
                tp1 = tp.clone()
                tp1[i] = 1
                tp0 = tp.clone()
                tp0[i] = 0
                offer = tpToPayments(tp1)[i]
                delay1 = tpToTotalDelay(tp1)
                delay0 = tpToTotalDelay(tp0)
                loss = loss + (1 - cdf(offer,order)) * delay1 + cdf(offer,order) * delay0

        loss = loss / len(tp_batch) / n
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(tp_batch),
                    len(tp_dataloader.dataset),
                    100.0 * batch_idx / len(tp_dataloader),
                    loss.item(),
                )
            )
    print("penalty:",penalty.item())
    
allBits = [torch.tensor(bits) for bits in itertools.product([0, 1], repeat=n)]

def test():
    model.eval()
    with torch.no_grad():
        costSharingLoss = 0
        dpLoss = 0
        nnLoss = 0
        heuristicLoss=0
        lenLoss= 0
        for (tp_batch,) in tp_dataloader_testing:

            for tp in tp_batch:
                costSharingLoss += costSharingDelay(tp)
                dpLoss += dpDelay(tp)
                nnLoss += tpToTotalDelay(tp)
                heuristicLoss+= heuristicDelay(tp)
            lenLoss+=len(tp_batch)
        costSharingLoss /= lenLoss
        dpLoss /= lenLoss
        nnLoss /= lenLoss
        heuristicLoss /= lenLoss
        #print(lenLoss)
        recordAndReport("NN", runningLossNN, nnLoss)
        recordAndReport("CS", runningLossCS, costSharingLoss)
        recordAndReport("DP", runningLossDP, dpLoss)
        recordAndReport("heuristic", runningLossHeuristic, heuristicLoss)
        print("DP:",dp[n, dpPrecision,0])
        #for i in range(n, 0, -1):
        #    print("Heuristic:",i,5*(1-dp_H[i, i, dpPrecision]))
        for i in range(n, 0, -1):
            print(
                    tpToPayments(
                            torch.tensor([0 if ii >= i else 1 for ii in range(n)], dtype=torch.float32)
                    )
                )
            
dpPrecision = 100 
def producedata(order):
    global  samplesJoint,tp_dataloader,tp_dataloader_testing,dp,decision,dp_H,decision_H
    dpPrecision = 100 
    if(order==stage[0]):
        print("loc",doublePeakLowMean, "scale",doublePeakStd)
        print("loc",doublePeakHighMean, "scale",doublePeakStd)
        signals = np.random.randint(2, size=(trainSize, n))
        samples1 = np.random.normal(
            loc=doublePeakLowMean, scale=doublePeakStd, size=(trainSize, n)
        )
        for i in range(trainSize):
            for j in range(n):
                while samples1[i, j] < 0 or samples1[i, j] > 1:
                    samples1[i, j] = np.random.normal(
                        loc=doublePeakLowMean, scale=doublePeakStd
                    )
        samples2 = np.random.normal(
            loc=doublePeakHighMean, scale=doublePeakStd, size=(trainSize, n)
        )
        for i in range(trainSize):
            for j in range(n):
                while samples2[i, j] < 0 or samples2[i, j] > 1:
                    samples2[i, j] = np.random.normal(
                        loc=doublePeakHighMean, scale=doublePeakStd
                    )
        samplesJoint = signals * samples1 - (signals - 1) * samples2
        tp_tensor = torch.tensor(samplesJoint, dtype=torch.float32)
        # tp_tensor = torch.tensor(np.random.rand(10000, n), dtype=torch.float32)
    elif(order==stage[1]):
        print("loc",normalloc, "scale",normalscale)
        samples1 = np.random.normal(
            loc=normalloc, scale=normalscale, size=(trainSize, n)
        )
        for i in range(trainSize):
            for j in range(n):
                while samples1[i, j] < 0 or samples1[i, j] > 1:
                    samples1[i, j] = np.random.normal(
                        loc=normalloc, scale=normalscale
                    )
        samplesJoint = samples1
        tp_tensor = torch.tensor(samplesJoint, dtype=torch.float32)
    elif(order==stage[2]):  
        print("uniformlow",uniformlow, "uniformhigh",uniformhigh)
        samples1 = np.random.uniform(
            uniformlow, uniformhigh, size=(trainSize, n)
        )
        for i in range(trainSize):
            for j in range(n):
                while samples1[i, j] < 0 or samples1[i, j] > 1:
                    samples1[i, j] = np.random.normal(
                        uniformlow, uniformhigh
                    )
        samplesJoint = samples1
        tp_tensor = torch.tensor(samplesJoint, dtype=torch.float32)
    
    tp_dataset = TensorDataset(tp_tensor[: int(trainSize * 0.3)])
    tp_dataset_testing = TensorDataset(tp_tensor[int(trainSize * 0.7) :])
    tp_dataloader = DataLoader(tp_dataset, batch_size=128, shuffle=True)
    tp_dataloader_testing = DataLoader(tp_dataset_testing, batch_size=256, shuffle=False)
    
    # howManyPpl left, money left, yes already
    dp = np.zeros([n + 1, dpPrecision + 1, n + 1])
    decision = np.zeros([n + 1, dpPrecision + 1, n + 1], dtype=np.uint8)
    # ppl = 0 left
    for yes in range(n + 1):
        for money in range(dpPrecision + 1):
            if money == 0:
                dp[0, 0, yes] = 0
            else:
                dp[0, money, yes] = yes# + 1.0
    for ppl in range(1,  n + 1):
        for yes in range(n + 1):
            for money in range(dpPrecision + 1):
                minSoFar = 1000000
                for offerIndex in range(money + 1):
                    offer = float(offerIndex) / dpPrecision
                    res = (1 - cdf(offer,order)) * dp[
                        ppl - 1, money - offerIndex, min(yes + 1, n)
                    ] + cdf(offer,order) * (1 + dp[ppl - 1, money, yes])
                    if minSoFar > res.item():
                        minSoFar = res.item()
                        decision[ppl, money, yes] = offerIndex
                dp[ppl, money, yes] = minSoFar
    
    print("dp",dp[n, dpPrecision, 0])
    
    # howManyPpl left, money left, yes already
    dp_H = np.zeros([n + 1 , n + 1, dpPrecision + 1])
    decision_H = np.zeros([n + 1 , n + 1, dpPrecision + 1], dtype=np.uint8)
    # ppl = 0 left
    for i in range(1,n+1):
        for money in range(dpPrecision + 1):
            if money == 0:
                dp_H[i, 0, 0] = 1
            else:
                offer = money / dpPrecision
                dp_H[i, 0, money] = 0#cdf(offer)# + 1.0
    for i in range(1,n+1):
        for ppl in range(1, i + 1):
            for money in range(dpPrecision + 1):
                maxSoFar = -1000000
                for offerIndex in range(money + 1):
                    offer = float(offerIndex) / dpPrecision
                    res = (1-cdf(offer,order)) * dp_H[
                         i, ppl - 1, money - offerIndex
                        ]
                    if maxSoFar < res.item():
                        maxSoFar = res.item()
                        decision_H[i, ppl, money] = offerIndex
                dp_H[i, ppl, money] = maxSoFar
                

In [6]:

for order in stage:
    producedata(order)

    # for mapping binary to payments before softmax
    model = nn.Sequential(
        nn.Linear(n, 100),
        nn.ReLU(),
        nn.Linear(100, 100),
        nn.ReLU(),
        nn.Linear(100, 100),
        nn.ReLU(),
        nn.Linear(100, n),
    )
    # optimizer = optim.SGD(model.parameters(), lr=lr)
    optimizer = optim.Adam(model.parameters(), lr=lr)


                
    #running
    for order1 in range(len(order1name)):
        print("Supervised Aim:",order,order1name[order1])

        runningLossNN = []
        runningLossCS = []
        runningLossDP = []
        runningLossHeuristic = []
        
        for batch_idx, (tp_batch,) in enumerate(tp_dataloader_testing):
            penalty = 0
            for bitsMoreOnes in allBits:
                for i in range(n):
                    if bitsMoreOnes[i] == 1:
                        bitsLessOnes = bitsMoreOnes.clone()
                        bitsLessOnes[i] = 0
                        penalty = penalty + sum(
                                torch.relu(
                                    bitsToPayments(bitsMoreOnes) - bitsToPayments(bitsLessOnes)
                                )
                            )
            loss = penalty * penaltyLambda
            for tp in tp_batch:
                    for i in range(n):
                        tp1 = tp.clone()
                        tp1[i] = 1
                        tp0 = tp.clone()
                        tp0[i] = 0
                        offer = tpToPayments(tp1)[i]
                        delay1 = tpToTotalDelay(tp1)
                        delay0 = tpToTotalDelay(tp0)
                        #loss = loss + (1 - cdf(offer)) * delay1 + cdf(offer) * delay0
                        loss = loss + (1 - cdf(offer,order)) * delay1 + cdf(offer,order) * delay0

            tp1 = tp.clone()
            tp1[0] = 1
            tp0 = tp.clone()
            tp0[0] = 0
            offer = tpToPayments(tp1)[0]
            break;
        #model=torch.load("save/pytorchNN=5dp1");
        #model.eval()
        for epoch in range(1, supervisionEpochs + 1):
#             print("distributionRatio",distributionRatio)
            if(order1==0):
                supervisionTrain(epoch, costSharingSupervisionRule)
            elif(order1==1):
                supervisionTrain(epoch, dpSupervisionRule)
            elif(order1==2):
                supervisionTrain(epoch, heuristicSupervisionRule)
            elif(order1==3):
                print("do nothing");

        test()

        for epoch in range(1, epochs + 1):
            train(epoch)
            test()

        savepath="save/pytorchNN=5all"+order+str(order1)
        torch.save(model, savepath);
        print("end")
        print()

loc 0.1 scale 0.1
loc 0.9 scale 0.1
dp 1.8651759624481201
Supervised Aim: twopeak costsharing
Train Epoch: 1 [0/3000 (0%)]	Loss: 0.000113
Train Epoch: 1 [2560/3000 (83%)]	Loss: 0.000001
Train Epoch: 2 [0/3000 (0%)]	Loss: 0.000002
Train Epoch: 2 [2560/3000 (83%)]	Loss: 0.000000
NN 1 : tensor(2.5627)
CS 1 : 2.562
DP 1 : 1.8656666666666666
heuristic 1 : 1.8286666666666667
DP: 1.8651759624481201
tensor([0.2003, 0.2001, 0.1999, 0.2002, 0.1995])
tensor([0.2496, 0.2500, 0.2501, 0.2502, 1.0000])
tensor([0.3330, 0.3336, 0.3334, 1.0000, 1.0000])
tensor([0.4997, 0.5003, 1.0000, 1.0000, 1.0000])
tensor([1., 1., 1., 1., 1.])
Train Epoch: 1 [0/3000 (0%)]	Loss: 2.509565
Train Epoch: 1 [2560/3000 (83%)]	Loss: 2.192957
penalty: 0.05495244264602661
NN 2 : tensor(2.3548)
CS 2 : 2.562
DP 2 : 1.8656666666666666
heuristic 2 : 1.8286666666666667
DP: 1.8651759624481201
tensor([0.0145, 0.1818, 0.5370, 0.0266, 0.2402])
tensor([0.0283, 0.2649, 0.6596, 0.0472, 1.0000])
tensor([0.0404, 0.2932, 0.6665, 1.0000, 1.00

tensor([0.2459, 0.2431, 0.2585, 0.2526, 1.0000])
tensor([0.3267, 0.3302, 0.3430, 1.0000, 1.0000])
tensor([0.5068, 0.4932, 1.0000, 1.0000, 1.0000])
tensor([1., 1., 1., 1., 1.])
Train Epoch: 2 [0/3000 (0%)]	Loss: 4.759440
Train Epoch: 2 [2560/3000 (83%)]	Loss: 4.805884
penalty: 0.0
NN 3 : tensor(4.7164)
CS 3 : 4.731333333333334
DP 3 : 4.779333333333334
heuristic 3 : 4.77
DP: 4.783679008483887
tensor([0.1997, 0.1996, 0.2086, 0.1975, 0.1946])
tensor([0.2487, 0.2434, 0.2526, 0.2554, 1.0000])
tensor([0.3323, 0.3317, 0.3359, 1.0000, 1.0000])
tensor([0.5078, 0.4922, 1.0000, 1.0000, 1.0000])
tensor([1., 1., 1., 1., 1.])
end

Supervised Aim: normal heuristic
Train Epoch: 1 [0/3000 (0%)]	Loss: 0.000004
Train Epoch: 1 [2560/3000 (83%)]	Loss: 0.000000
Train Epoch: 2 [0/3000 (0%)]	Loss: 0.000000
Train Epoch: 2 [2560/3000 (83%)]	Loss: 0.000000
NN 1 : tensor(4.7330)
CS 1 : 4.731333333333334
DP 1 : 4.779333333333334
heuristic 1 : 4.77
DP: 4.783679008483887
tensor([0.2001, 0.1997, 0.2022, 0.1988, 0.1993

tensor([0.4868, 0.5132, 1.0000, 1.0000, 1.0000])
tensor([1., 1., 1., 1., 1.])
Train Epoch: 1 [0/3000 (0%)]	Loss: 1.385183
Train Epoch: 1 [2560/3000 (83%)]	Loss: 1.508479
penalty: 0.0
NN 2 : tensor(1.4063)
CS 2 : 1.4133333333333333
DP 2 : 1.6383333333333334
heuristic 2 : 1.533
DP: 1.6252939701080322
tensor([0.1972, 0.2015, 0.2027, 0.2040, 0.1946])
tensor([0.2372, 0.2521, 0.2532, 0.2575, 1.0000])
tensor([0.3113, 0.3560, 0.3328, 1.0000, 1.0000])
tensor([0.4826, 0.5174, 1.0000, 1.0000, 1.0000])
tensor([1., 1., 1., 1., 1.])
Train Epoch: 2 [0/3000 (0%)]	Loss: 1.638316
Train Epoch: 2 [2560/3000 (83%)]	Loss: 1.475922
penalty: 0.0
NN 3 : tensor(1.4059)
CS 3 : 1.4133333333333333
DP 3 : 1.6383333333333334
heuristic 3 : 1.5330000000000001
DP: 1.6252939701080322
tensor([0.1965, 0.2028, 0.2005, 0.2053, 0.1948])
tensor([0.2345, 0.2581, 0.2497, 0.2577, 1.0000])
tensor([0.3039, 0.3672, 0.3289, 1.0000, 1.0000])
tensor([0.4761, 0.5239, 1.0000, 1.0000, 1.0000])
tensor([1., 1., 1., 1., 1.])
end

